# tochtext 튜토리얼 - 영어
## 1. 데이터셋 분리  
### 데이터 다운로드

In [2]:
import urllib.request
import pandas as pd

In [16]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="dataset\IMDb_Reviews.csv")


('dataset\\IMDb_Reviews.csv', <http.client.HTTPMessage at 0x2419ecb31c0>)

In [18]:
df = pd.read_csv('dataset\IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [19]:
print('전체 샘플의 개수 : {}'.format(len(df)))


전체 샘플의 개수 : 50000


### 학습데이터와 테스트 데이터로 분리

In [20]:
train_df = df[:25000]
test_df = df[25000:]

In [22]:
train_df.to_csv("./dataset/train_data.csv", index=False)
test_df.to_csv("./dataset/test_data.csv", index=False)

## 2. 필드 정의

In [23]:
from torchtext import data


torchtext.data는 필드(Field)라는 도구를 제공하여 앞으로 어떤 전처리를 할 것인지 정의할 수 있음

In [45]:
# 실제 텍스트를 위한 TEXT 객체
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  #fix_length=20
                  fix_length=150 )

# 레이블 데이터를 위한 LABEL 객체
LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

- sequential : 시퀀스 데이터 여부. (True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
- is_target : 레이블 데이터 여부. (False가 기본값)
- fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

## 3. 데이터셋 만들기
- TabularDataset은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행
- 소문자화 같은 기본적인 전처리도 함께 이루어짐

In [27]:
from torchtext.data import TabularDataset

In [46]:
train_data, test_data = TabularDataset.splits(
        path='.', train='./dataset/train_data.csv', test='./dataset/test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

- path : 파일이 위치한 경로.
- format : 데이터의 포맷.
- fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
- skip_header : 데이터의 첫번째 줄은 무시.

In [47]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


vars()를 통해서 주어진 인덱스의 샘플을 확인할 수 있음

In [48]:
print(vars(train_data[0]))


{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

In [49]:
print(train_data.fields.items())


dict_items([('text', <torchtext.data.field.Field object at 0x00000242050B2670>), ('label', <torchtext.data.field.Field object at 0x00000242050B2F70>)])


## 4. Vocabulary 만들기
정의한 필드에 .build_vocab() 도구를 사용하면 단어 집합을 생성

In [50]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


- min_freq : 단어 집합에 추가 시 단어의 최소 등장 빈도 조건을 추가.
- max_size : 단어 집합의 최대 크기를 지정

생성된 단어 집합 내의 단어들은 .stoi를 통해서 확인 가능

In [ ]:
print(TEXT.vocab.stoi)


## 데이터로더
- 데이터로더는 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할
- 토치텍스트에서는 Iterator를 사용하여 데이터로더를 만듬

In [38]:
from torchtext.data import Iterator


배치 크기를 5로 정함

In [52]:
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 5000
테스트 데이터의 미니 배치 수 : 5000


훈련 데이터와 테스트 데이터 모두 미니 배치의 수가 5,000개인데 이는 25,000개의 샘플을 배치 크기 5씩 묶어주었기 때문

첫번째 미니 배치를 받아와서 batch라는 변수에 저장

In [53]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(type(batch))
print(batch.text) #첫번째 미니 배치의 text 필드를 호출

<class 'torchtext.data.batch.Batch'>
tensor([[   9,   83,  441,   48,  259,  395, 3646,   10,   20,    7,    2,  246,
          404,    5, 1799, 2296,   19,  138,   48,   14,  306,    0,    0,   13,
          159,  115,   43,  418,   19,    3,  265,    5, 2672, 5307,   18,    0,
           53,  411,  743,   10,  118,   43, 1399,    4, 1557,  905, 4105,  294,
          305,    0, 5509,   12,   21,   59,  768,    7, 3280,  405, 9791,   13,
          159,   98,    2,   59,  313,    2,    0, 1471, 1545,   19,    2, 2156,
          246,  127,    5,   36,   84,    7,   12,   59,  974,   40,   38,   89,
           75, 1057,    6,  148,   10, 3084,   29,  225,    0,   95,   11,   19,
            0,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    

- 각 샘플의 길이는 20의 길이를 가지는데, 이는 앞서 초기에 필드를 정의할 때 fix_length를 20으로 정해주었기 때문 => 하나의 미니 배치의 크기는 (배치 크기 × fix_length)
- 단어는 단어 집합에서 정해진대로 각 단어에 맵핑되는 고유한 정수로 변환된 상태
- 샘플의 중간, 중간에는 숫자 0이 존재하는데 이는  < unk > 토큰의 번호로 단어 집합에 포함되지 못한 단어들은 < unk >라는 토큰으로 변환

## < pad >토큰이 사용되는 경우
-  fix_length를 20이 아니라 150으로 정의하고 진행
- 샘플의 뒷 부분이 < pad >의 번호였던 숫자 1로 채워짐
- 서로 다른 길이의 샘플들을 동일한 길이로 맞춰추는 padding 작업이 진행됨

In [54]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text[0]) # 첫번째 미니배치 중 첫번째 샘플

tensor([ 956,    2,  680, 2068,   23,   95,    5,    7,   10, 5622,    5,    2,
           0, 2244,   31,    3, 2980,    0,    0,    0,    4,   21, 1141,    0,
        5481,    0,    0,  560,    6,    0,    3, 2312,  418,   19, 1209, 5212,
           0,   13,  721, 4695,   25,   16,   81,  389,    4,    3,    0,  733,
           5,    2,  232,    5, 3615, 5084,  102, 4978,    8,    2,   51, 2119,
         127,    5,    2,    0, 6042,    5,    2,    0,    5,    0,    8,  293,
         575,  223,   11,   14,    2,   59,    0,   25,    6, 1310,    3,  119,
         551,    0,   13,    0,    0,    7,    2,  936,    0,   35,  244,    6,
           2,  187,  752,  979,    6,  152,  430,   32,  771,  111,    6,   90,
         330,   50,   33,  549,  128,   21, 5306,   18,  560,  607,    2,  752,
           7,    3, 1386,  327,   17,   60, 3435,    0,   16,   21,    0,    0,
          31,    3, 1141,    0,    0,    0,   27,  253,  143,    3,  265,    5,
           0, 3080,   12,  545,  101,   

## batch_first
토치텍스트에서 필드정의할 때 batch_first를 True로 한 경우와 False를 한 경우를 비교
- True : torch.Size([5, 20])
- False : torch.Size([20, 5]) => 미니 배치의 크기는 (fix_length × 배치 크기)